In [4]:
!pip install transformers

In [5]:
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 MB 3.1 MB/s eta 0:00:00:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 5.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 6.9 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 629.4 kB/s eta 0:00:00 0:00:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.8.0
    Uninstalling typing_extensions-4.8.0:
      Successfully uninstalled typing_extensions-4.8.0
  Attempting uninstall: sympy
    Found existing installation: sympy 1.12
    Uninstalling sympy-1.12:
      Successfully uninstalled sympy-1.12


In [6]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegressionModel
from pyspark.ml.feature import StringIndexerModel
from pyspark.ml import PipelineModel
from pyspark.ml.linalg import Vectors
from pyspark.sql import Row
from pyspark.ml.functions import array_to_vector

In [7]:
# ------------------------------------------------------------
# 1️⃣ Setup Spark Session
# ------------------------------------------------------------
spark = SparkSession.builder \
    .appName("RAID-INFERENCE") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

print("✅ Spark session aktif:", spark)

✅ Spark session aktif: <pyspark.sql.session.SparkSession object at 0x7cfebc257190>


In [ ]:
# ------------------------------------------------------------
# 2️⃣ Load Pretrained Model dari HDFS
# ------------------------------------------------------------
MODEL_PATH = "hdfs://namenode:8020/user/raid/model-sbert-lr_human-gpt4"

print(f"📂 Memuat model dari: {MODEL_PATH}")
model = PipelineModel.load(MODEL_PATH)
print("✅ Model berhasil dimuat.")
model

📂 Memuat model dari: hdfs://namenode:8020/user/raid/model-sbert-lr_human-gpt4


In [ ]:
# ------------------------------------------------------------
# 3️⃣ Siapkan Sentence Transformer MiniLM
# ------------------------------------------------------------
print("📥 Memuat model MiniLM untuk ekstraksi fitur...")

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
encoder = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
encoder.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
encoder.to(device)